1. Create Delta Tables Using 3 Methods

In [0]:
dbutils.fs.cp("file:/Workspace/Shared/sales_dataa.csv","dbfs:/FileStore/sales_dataa.csv")

sales_df=spark.read.format("csv").option("header","true").load("/FileStore/sales_dataa.csv")
sales_df.show()


+-------+----------+----------+--------+--------+-----+
|OrderID| OrderDate|CustomerID| Product|Quantity|Price|
+-------+----------+----------+--------+--------+-----+
|   1001|2024-01-15|      C001|Widget A|      10|25.50|
|   1002|2024-01-16|      C002|Widget B|       5|15.75|
|   1003|2024-01-16|      C001|Widget C|       8|22.50|
|   1004|2024-01-17|      C003|Widget A|      15|25.50|
|   1005|2024-01-18|      C004|Widget D|       7|30.00|
|   1006|2024-01-19|      C002|Widget B|       9|15.75|
|   1007|2024-01-20|      C005|Widget C|      12|22.50|
|   1008|2024-01-21|      C003|Widget A|      10|25.50|
+-------+----------+----------+--------+--------+-----+



In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DateType

# Define the schema
customer_schema = StructType([
    StructField("CustomerID", StringType(), True),
    StructField("CustomerName", StringType(), True),
    StructField("Region", StringType(), True),
    StructField("SignupDate", StringType(), True)  # Keep as StringType to convert later
])

# Load the JSON with the defined schema
customer_df = spark.read.format("json").schema(customer_schema).load("/FileStore/cust_data.json")


In [0]:
#DataFrame as a Delta Table
sales_df.write.format("delta").mode("overwrite").save("/delta/sales_dataa_delta")



In [0]:
#Load the customer_data.json file into a DataFrame.

dbutils.fs.cp("file:/Workspace/Shared/cust_data.json", "dbfs:/FileStore/cust_data.json")

# Load customer data from JSON
customer_df = spark.read.option("multiline", "true").json("/FileStore/cust_data.json")

customer_df.show()


+----------+-------------+------+----------+
|CustomerID| CustomerName|Region|SignupDate|
+----------+-------------+------+----------+
|      C001|     John Doe| North|2022-07-01|
|      C002|   Jane Smith| South|2023-02-15|
|      C003|Emily Johnson|  East|2021-11-20|
|      C004|Michael Brown|  West|2022-12-05|
|      C005|  Linda Davis| North|2023-03-10|
+----------+-------------+------+----------+



In [0]:
#Cust table as a delta table
# Write customer data as a Delta table
customer_df.write.format("delta").mode("overwrite").save("/delta/customer_data_delta")

customer_delta_df = spark.read.format("delta").load("/delta/customer_data_delta")
customer_delta_df.show()



+----------+-------------+------+----------+
|CustomerID| CustomerName|Region|SignupDate|
+----------+-------------+------+----------+
|      C001|     John Doe| North|2022-07-01|
|      C002|   Jane Smith| South|2023-02-15|
|      C003|Emily Johnson|  East|2021-11-20|
|      C004|Michael Brown|  West|2022-12-05|
|      C005|  Linda Davis| North|2023-03-10|
+----------+-------------+------+----------+



In [0]:
# Create a sample DataFrame
sample_data = [("John", 30), ("Jane", 25), ("Sam", 35)]
columns = ["Name", "Age"]
sample_df = spark.createDataFrame(sample_data, columns)

# Write the DataFrame as a Parquet file
sample_df.write.mode("overwrite").parquet("/FileStore/sample_parquet_file.parquet")


In [0]:
# Load the Parquet file
parquet_df = spark.read.parquet("/FileStore/sample_parquet_file.parquet")

# Convert the Parquet file to a Delta Table
parquet_df.write.format("delta").mode("overwrite").save("/delta/sample_parquet_to_delta")

In [0]:
dbutils.fs.cp("file:/Workspace/Shared/new_sales_data.csv", "dbfs:/FileStore/new_sales_data.csv")

True

2. Data Management

In [0]:
dbutils.fs.cp("file:/Workspace/Shared/new_sales_data.csv", "dbfs:/FileStore/new_sales_data.csv")
# Load new sales data into DataFrame
new_sales_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("/FileStore/new_sales_data.csv")
new_sales_df.show()


+-------+----------+----------+--------+--------+-----+
|OrderID| OrderDate|CustomerID| Product|Quantity|Price|
+-------+----------+----------+--------+--------+-----+
|   1009|2024-01-22|      C006|Widget E|      14| 20.0|
|   1010|2024-01-23|      C007|Widget F|       6| 35.0|
|   1002|2024-01-16|      C002|Widget B|      10|15.75|
+-------+----------+----------+--------+--------+-----+



In [0]:
from delta.tables import *

# Load existing Delta table
delta_table =spark.read.format("delta").load("/delta/sales_dataa_delta")
delta_new_sales=spark.read.format("delta").load("/delta/sales_dataa_delta")
# Create temporary views for SQL operations
delta_table.createOrReplaceTempView("delta_sales_data")
delta_new_sales.createOrReplaceTempView("new_sales_data")

In [0]:
# Merge new sales data into existing Delta Table

spark.sql("""
          MERGE INTO delta_sales_data AS target
          USING new_sales_data AS source
          ON target.OrderID = source.OrderID
          WHEN MATCHED THEN UPDATE SET 
              target.OrderDate = source.OrderDate, 
              target.CustomerID = source.CustomerID,   
              target.Product = source.Product,
              target.Quantity = source.Quantity, 
              target.Price = source.Price
          WHEN NOT MATCHED THEN INSERT (OrderID, OrderDate, CustomerID, Product, Quantity, Price)
          VALUES (source.OrderID, source.OrderDate, source.CustomerID, source.Product, source.Quantity, source.Price)
""")


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

3. Optimize Delta Table

In [0]:
%sql
OPTIMIZE delta.`/delta/sales_dataa_delta` ZORDER BY (CustomerID)



path,metrics
dbfs:/delta/sales_dataa_delta,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 1792), 0, List(0, 0), 0, null), 0, 1, 1, false, 0, 0, 1726211553584, 1726211554094, 4, 0, null, List(0, 0), 6, 6, 0, 0, null)"


4. Advanced Features


In [0]:
%sql
DESCRIBE HISTORY delta.`/delta/sales_dataa_delta`



version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
16,2024-09-13T09:47:21Z,7670452412022086,azuser2139_mml.local@techademy.com,MERGE,"Map(predicate -> [""(OrderID#12823 = OrderID#12835)""], matchedPredicates -> [{""actionType"":""update""}], statsOnLoad -> false, notMatchedBySourcePredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(387350043725306),0911-070944-tra6x1ll,15,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, numTargetBytesAdded -> 1794, numTargetBytesRemoved -> 1794, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 10, executionTimeMs -> 1517, numTargetRowsInserted -> 0, numTargetRowsMatchedDeleted -> 0, numTargetDeletionVectorsUpdated -> 0, scanTimeMs -> 559, numTargetRowsUpdated -> 10, numOutputRows -> 10, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 10, numTargetFilesRemoved -> 1, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 813)",null,Databricks-Runtime/14.3.x-photon-scala2.12
15,2024-09-13T09:41:03Z,7670452412022086,azuser2139_mml.local@techademy.com,WRITE,"Map(mode -> Overwrite, statsOnLoad -> false, partitionBy -> [])",null,List(387350043725306),0911-070944-tra6x1ll,14,WriteSerializable,false,"Map(numFiles -> 1, numOutputRows -> 10, numOutputBytes -> 1794)",null,Databricks-Runtime/14.3.x-photon-scala2.12
14,2024-09-13T07:39:16Z,7670452412022086,azuser2139_mml.local@techademy.com,VACUUM END,Map(status -> COMPLETED),null,List(387350043725306),0911-070944-tra6x1ll,13,SnapshotIsolation,true,"Map(numDeletedFiles -> 3, numVacuumedDirectories -> 1)",null,Databricks-Runtime/14.3.x-photon-scala2.12
13,2024-09-13T07:39:11Z,7670452412022086,azuser2139_mml.local@techademy.com,VACUUM START,"Map(retentionCheckEnabled -> false, defaultRetentionMillis -> 604800000, specifiedRetentionMillis -> 0)",null,List(387350043725306),0911-070944-tra6x1ll,12,SnapshotIsolation,true,"Map(numFilesToDelete -> 3, sizeOfDataToDelete -> 3380)",null,Databricks-Runtime/14.3.x-photon-scala2.12
12,2024-09-13T07:23:07Z,7670452412022086,azuser2139_mml.local@techademy.com,OPTIMIZE,"Map(predicate -> [], zOrderBy -> [], batchId -> 0, auto -> true)",null,List(387350043725306),0911-070944-tra6x1ll,11,SnapshotIsolation,false,"Map(numRemovedFiles -> 2, numRemovedBytes -> 3287, p25FileSize -> 1794, numDeletionVectorsRemoved -> 1, minFileSize -> 1794, numAddedFiles -> 1, maxFileSize -> 1794, p75FileSize -> 1794, p50FileSize -> 1794, numAddedBytes -> 1794)",null,Databricks-Runtime/14.3.x-photon-scala2.12
11,2024-09-13T07:23:05Z,7670452412022086,azuser2139_mml.local@techademy.com,MERGE,"Map(predicate -> [""(OrderID#8544 = OrderID#8573)""], matchedPredicates -> [{""actionType"":""update""}], statsOnLoad -> false, notMatchedBySourcePredicates -> [], notMatchedPredicates -> [{""actionType"":""insert""}])",null,List(387350043725306),0911-070944-tra6x1ll,10,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 1, numTargetBytesAdded -> 1495, numTargetBytesRemoved -> 0, numTargetDeletionVectorsAdded -> 1, numTargetRowsMatchedUpdated -> 3, executionTimeMs -> 2075, numTargetRowsInserted -> 0, numTargetRowsMatchedDeleted -> 0, numTargetDeletionVectorsUpdated -> 0, scanTimeMs -> 671, numTargetRowsUpdated -> 3, numOutputRows -> 3, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -> 3, numTargetFilesRemoved -> 0, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 1148)",null,Databricks-Runtime/14.3.x-photon-scala2.12
10,2024-09-13T07:17:34Z,7670452412022086,azuser2139_mml.local@techademy.com,VACUUM END,Map(status -> COMPLETED),null,List(387350043725306),0911-070944-tra6x1ll,9,SnapshotIsolation,true,"Map(num

In [0]:
%sql
VACUUM delta.`/delta/sales_dataa_delta` 



path
dbfs:/delta/sales_dataa_delta


5. Hands-on Exercises

In [0]:
# Query a historical version of the Delta table
historical_sales_df = spark.read.format("delta").option("versionAsOf", 2).load("/delta/sales_dataa_delta")
historical_sales_df.show()


+-------+----------+----------+--------+--------+-----+
|OrderID| OrderDate|CustomerID| Product|Quantity|Price|
+-------+----------+----------+--------+--------+-----+
|   1001|2024-01-15|      C001|Widget A|      10|25.50|
|   1002|2024-01-16|      C002|Widget B|       5|15.75|
|   1003|2024-01-16|      C001|Widget C|       8|22.50|
|   1004|2024-01-17|      C003|Widget A|      15|25.50|
|   1005|2024-01-18|      C004|Widget D|       7|30.00|
|   1006|2024-01-19|      C002|Widget B|       9|15.75|
|   1007|2024-01-20|      C005|Widget C|      12|22.50|
|   1008|2024-01-21|      C003|Widget A|      10|25.50|
+-------+----------+----------+--------+--------+-----+



In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Sample data
data = [("John Doe", 30), ("Jane Doe", 25)]

# Define schema
schema = StructType([
    StructField("Name", StringType(), True),
    StructField("Age", IntegerType(), True)
])

# Create DataFrame
sales_df = spark.createDataFrame(data, schema)


In [0]:
# Enforce schema when writing to a Delta Table
sales_df.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("/delta/sales_data")

In [0]:
%sql
VACUUM delta.`/delta/sales_dataa_delta` 

path
dbfs:/delta/sales_dataa_delta
